In [4]:
# Imports
import logging
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas
import scipy
from shutil import copyfile
from spectral.io import envi
from scipy.interpolate import splrep, splev, interp1d

import isofit
from isofit.__main__ import env
from isofit.core.fileio import IO
from isofit.core.forward import ForwardModel
from isofit.inversion.inverse import Inversion
from isofit.configs import configs
from isofit.core.geometry import Geometry

from functions import make_template_and_config
from functions import make_rois
from functions import sample_points
from functions import make_lut

# Enable the ISOFIT logger
logging.getLogger().setLevel(logging.INFO)
# logging.getLogger().setLevel(logging.ERROR)

In [7]:
str(env.path('data') / 'emit_noise.txt')

'/Users/bgreenbe/.isofit/data/emit_noise.txt'

In [8]:
# Define all constants
constants = {
    'working_root': '/Users/bgreenbe/Projects/CalibrationTutorial/Luts',
    'isofit_path': os.path.dirname(isofit.__file__),
    'lut_base': '/Users/bgreenbe/Projects/CalibrationTutorial/Luts',
    'aerosol_tpl_path': str(env.path('data') / 'aerosol_template.json'),
    'earth_sun_distance_file': str(env.path('data') / 'earth_sun_distance.txt'),
    'emulator_file': str(env.path("srtmnet", key="srtmnet.file")),
    'engine_base_dir':  str(env.path('data')),
    'emulator_aux_file': str(env.path("srtmnet", key="srtmnet.aux")),
    'engine_name': 'sRTMnet',
    'irradiance_file': str(env.path('examples') /'20151026_SantaMonica/data/prism_optimized_irr.dat'),
    'wavelength_file': '/Users/bgreenbe/Projects/MultiSurface/Examples/EMIT/202409_tests/dev_sample_emit20240727T212906/data/wavelengths.txt',
    'surface_path': '/Users/bgreenbe/Data/EMIT/Surfaces/surface_20240103.mat',
    'inversion_windows': [[380.0, 1325.0], [1435, 1770.0], [1965.0, 2500.0]],
    'parametric_noise_file': str(env.path('data') / 'emit_noise.txt'),
    'channelized_radiometric_uncertainty_file': str(env.path('data') / 'emit_osf_uncertainty.txt'),
    'model_discrepancy_file': str(env.path('data') / 'emit_model_discrepancy.mat'),
}
constants['h2o_lut_grid'] = list(np.linspace(.05, 5.0, 21))

In [3]:
# Define file lists to run through. If .nc passed, will pull loc from obs.
file_list = [
    {
        'rdn': '/Users/bgreenbe/Projects/CalibrationTutorial/emit20220818T205803/EMIT_L1B_RAD_001_20220818T205803_2223014_006.nc',
        'obs': '/Users/bgreenbe/Projects/CalibrationTutorial/emit20220818T205803/EMIT_L1B_OBS_001_20220818T205803_2223014_006.nc',
        'loc': ''
    },
    {
        'rdn': '/Users/bgreenbe/Projects/MultiSurface/Examples/EMIT/202409_tests/multi_sample_emit20240727T212906/emit20240727T212906_rdn',
        'obs': '/Users/bgreenbe/Projects/MultiSurface/Examples/EMIT/202409_tests/multi_sample_emit20240727T212906/emit20240727T212906_obs',
        'loc': '/Users/bgreenbe/Projects/MultiSurface/Examples/EMIT/202409_tests/multi_sample_emit20240727T212906/emit20240727T212906_loc',
    },
]

In [ ]:
# Optional: Make the scene-wide luts each input file.
# Otherwise, program will run LUTs that are only appropriate for the respective samples
scene_lut = True
if scene_lut:
    for i, files in enumerate(file_list):
        fid = files['rdn'].split('/')[-2]
        # Set file lut dir and file
        lut_base = constants['lut_base']
        fid_lut_dir = os.path.join(lut_base, fid)
        
        if not os.path.exists(fid_lut_dir):
            os.makedirs(fid_lut_dir)
    
        # Make the isofit and modtran configs
        tmpl_outpath, config_outpath = make_template_and_config(
            obs,
            loc,
            fid,
            lut_directory=fid_lut_dir,
            **constants,
        )
        make_lut(config_outpat)

In [4]:
# if you are sampling pre-defined points. yx is a list of tuples to sample. Either (lat, lon) pairs or (row, col) pairs. 
# These lists can be strung together to match the file_list structure
# Lat-lon example. Will throw general error if one is outside image boundaries.
yx = [
    [
        (35.50, -121.07),
        (35.50, -121.04),
    ],    
    [
        (35.50, -121.07),
        (35.50, -121.04),
    ],
]

# Row-col example
yx = [
    [
        (30, 39),
        (61, 12),
    ],
    [
        (30, 39),
        (61, 12),
    ],
]

In [5]:
# Generate ROIs and configs. 
# Note: This current method will make individual LUT files for the geometry of the rois covered in the sample.
matplotlib.use('TkAgg')

roi_df = pandas.DataFrame()
for i, files in enumerate(file_list):
    rdn_path = files['rdn']
    obs_path = files['obs']
    loc_path = files['loc']
    #df, rdn, obs, loc = make_rois(rdn_path, obs_path, loc_path)
    df, rdn, obs, loc = sample_points(yx[i], rdn_path, obs_path, loc_path, mode='index')

    fid = files['rdn'].split('/')[-2]
    df['fid'] = fid

    if not i:
        if len(rdn.shape) > 1:
            rdn_full = rdn
            obs_full = obs
            loc_full = loc
        else:
            rdn_full = np.reshape(rdn, (1, len(rdn)))
            obs_full = np.reshape(obs, (1, len(obs)))
            loc_full = np.reshape(loc, (1, len(loc)))
    else:
        df['row_start'] += roi_df.iloc[-1]['row_stop']
        df['row_stop'] += roi_df.iloc[-1]['row_stop']
        
        rdn_full = np.vstack((rdn_full, rdn))
        obs_full = np.vstack((obs_full, obs))
        loc_full = np.vstack((loc_full, loc))

    # Set file lut dir and file
    lut_base = constants['lut_base']
    fid_lut_dir = os.path.join(lut_base, fid)
    
    if not os.path.exists(fid_lut_dir):
        os.makedirs(fid_lut_dir)

    # Make the isofit and modtran configs
    tmpl_outpath, config_outpath = make_template_and_config(
        obs,
        loc,
        fid,
        lut_directory=fid_lut_dir,
        **constants,
    )
    print(config_outpath)
    df['isofit_config'] = config_outpath
    df['modtran_template'] = tmpl_outpath
    
    roi_df = pandas.concat([roi_df, df]).reset_index(drop=True)

INFO:root:Loading Climatology
INFO:root:Climatology Loaded.  Aerosol State Vector:
{'AOT550': {'bounds': [0.001, 1.0], 'scale': 1, 'init': 0.1009, 'prior_sigma': 10.0, 'prior_mean': 0.1009}}
Aerosol LUT Grid:
{'AOT550': [0.001, 0.1009, 0.2008, 0.3007, 0.4006, 0.5005, 0.6004, 0.7003, 0.8002, 0.9001, 1.0]}
Aerosol model path:/Users/bgreenbe/.isofit/data/aerosol_model.txt
INFO:root:Loading Climatology
INFO:root:Climatology Loaded.  Aerosol State Vector:
{'AOT550': {'bounds': [0.001, 1.0], 'scale': 1, 'init': 0.1009, 'prior_sigma': 10.0, 'prior_mean': 0.1009}}
Aerosol LUT Grid:
{'AOT550': [0.001, 0.1009, 0.2008, 0.3007, 0.4006, 0.5005, 0.6004, 0.7003, 0.8002, 0.9001, 1.0]}
Aerosol model path:/Users/bgreenbe/.isofit/data/aerosol_model.txt


gmtime: 20.966666666666665
mean_path_km: 418.888427734375
Mean Sensor Azimuth: 89.73745727539062
Mean Sensor Zenith: 8.946113586425781
Mean Solar Zenith: 30.550724029541016
Mean Solar Azimuth: 209.88902282714844
Mean Latitude: 40.44458444674301
Mean Longitude: -118.51042383507894
Mean Elevation (km): 1650.7172076149784
Mean altitude (km): 99
Sensor Zenith Grid: [8.946114]
Solar Zenith Grid: [30.550724]
Relative Azimuth Grid: [120.151566]
Elevation Grid (km): [1650.7172076149784]
/Users/bgreenbe/Projects/CalibrationTutorial/Luts/isofit_config_emit20220818T205803.json
gmtime: 21.483333333333334
mean_path_km: 419.99578857421875
Mean Sensor Azimuth: 79.0981216430664
Mean Sensor Zenith: 8.616144180297852
Mean Solar Zenith: 23.96978759765625
Mean Solar Azimuth: 231.45326232910156
Mean Latitude: 35.49990463256836
Mean Longitude: -121.05504608154297
Mean Elevation (km): 38.36846160888672
Mean altitude (km): 99
Sensor Zenith Grid: [8.616144]
Solar Zenith Grid: [23.969788]
Relative Azimuth Grid:

In [6]:
# Run inversion for each ROI
for i, row in roi_df.iterrows():
    config = configs.create_new_config(row['isofit_config'])
    config.get_config_errors()
    
    fm = ForwardModel(config)
    iv = Inversion(config, fm)

    esd_file = config.forward_model.radiative_transfer.radiative_transfer_engines[0].earth_sun_distance_file
    esd = IO.load_esd(esd_file)

    meas_roi = np.nanmean(rdn_full[row['row_start']:row['row_stop']], axis=0)
    obs_roi = np.nanmean(obs_full[row['row_start']:row['row_stop']], axis=0)
    loc_roi = np.nanmean(loc_full[row['row_start']:row['row_stop']], axis=0)
    
    geom = Geometry(
        obs=obs_roi,
        loc=loc_roi,
        esd=esd,
    )
    states = iv.invert(meas_roi, geom)

    if not i:
        roi_states = np.zeros((len(roi_df), states.shape[1]))

    roi_states[i, :] = states[-1]

INFO:root:Loading config file: /Users/bgreenbe/Projects/CalibrationTutorial/Luts/isofit_config_emit20220818T205803.json
INFO:root:Checking config sections for configuration issues
ERROR:root:Instrument config file: /Users/bgreenbe/Github/isofit/data/emit_noise.txt not found


AttributeError: Configuration error(s) found.  See log for details.

In [ ]:
%matplotlib inline
# Plotting

fig, axs = plt.subplots(2, 1, sharex=True)

wl = fm.RT.wl
for i, row in roi_df.iterrows():
    state = roi_states[i, :]
    rdn = np.nanmean(rdn_full[row['row_start']:row['row_stop']], axis=0)
    
    axs[0].plot(wl, rdn)
    axs[1].plot(wl, state[fm.idx_surf_rfl], label=f'roi_{i}')

plt.legend()
axs[1].set_xlabel('Wavelength (nm)')
axs[0].set_ylabel('Radiance')
axs[1].set_ylabel('Reflectance')